<div class="alert alert-block alert-info">
<center><img src='webscraping.png'/></center>
<h1><center>WEB SCRAPING PYTHON 2.0 PLACES</center></h1>
    <hr>
    <h3><center>Description:</center></h3>
    <p>This is a second way to make a WS (WebScraping) on TripAdvisor web page to get places comments. Actually, there are many ways to create a web crawler but to be practice, we will use Selenium to interact on the entire web page making clicks and moving through the links</p>
</div>

<div class="alert alert-block alert-warning">
<h3><center>Imports</center></h3>
</div>

In [1]:
###
# Libraries system or commonly called: Utils
###
import os
import time
import datetime
from datetime import date
import csv
import random
print("Libraries 'Utils' ready...")
###
# Selenium
###
try:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
    print("Selenium libraries ready...")
except ImportError as error:
    print("There is not a Module named 'Selenium' installed, please run the next cell...")
###
# urllib and BautifulSoup
###
try:
    import urllib
    from bs4 import BeautifulSoup
    print("BeautifulSoup and urllib libraries ready...")
except ImportError as error:
    print("There is not a Module named 'BeautifulSoup' installed, please run the next cell..")
###
# Forex -> Procing of the currencies
###
try:
    from forex_python.converter import CurrencyRates
    print("CurrencyRates libraries ready...")
except ImportError as error:
    print("There is not a Module named 'CurrencyRates' installed, please run the next cell...")
###
# Pandas
###
try:
    import pandas as pd
    print("Pandas libraries ready...")
except ImportError as error:
    print("There is not a Module named 'pandas' installed, please tun the next cell...")

Libraries 'Utils' ready...
Selenium libraries ready...
BeautifulSoup and urllib libraries ready...
CurrencyRates libraries ready...
Pandas libraries ready...


<div class="alert alert-warning" role="alert">
  <b>In case of Selenium is not installed</b>, run the next cell. Important!, althoug you had been installed Selenium that not mean that all Selenium package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install forex-python

<div class="alert alert-warning" role="alert">
  <b>In case of BeautifulSoup is not installed</b>, run the next cell. Important!, althoug you had been installed BeautifulSoup that not mean that all BeautifulSoup package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install beautifulsoup4 

<div class="alert alert-warning" role="alert">
  <b>In case of forex-python is not installed</b>, run the next cell. Important!, althoug you had been installed forex-python that not mean that all forex-python package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install forex-python

<div class="alert alert-warning" role="alert">
  <b>In case of Pandas is not installed</b>, run the next cell. Important!, althoug you had been installed Pandas that not mean that all Pandas package with all extentios has been downloaded, please, if you identify that the error handling continuous, please add a new cell bellow here and make the pip instalation.
</div>

In [ ]:
import sys
!{sys.executable} -m pip install pandas

<div class="alert alert-block alert-danger">
<h3><center>Importan!</center></h3>
    <hr>
    <p>Remember, <b>DON´T RUN</b> the pip install with -U beacuse of you will breake your Anaconda Enviroment due to uncompatible versions of the different libraries. Just RUN the pip install <b>WITHOUT -U</b> argument.</p>
</div>

<div class="alert alert-block alert-warning">
<h3><center>Methods</center></h3>
</div>

In [2]:
###
# Methods for handling SELENIUM
###
def connect(url):
    """This method will connect to the web site using selenium
       Parameters:
       ----------
           url: str, the url of the web page to scrap
       Returns:
       -------
           driver: ChromeDriver, the browser 
       Example:
       -------
           >>> connect('http://www.somepage.com')"""
    chromeOptions = webdriver.ChromeOptions()
    #chromeOptions.add_argument("--incognito")
    driver = webdriver.Chrome("chromedriver.exe",options=chromeOptions)
    driver.get(url) 
    wait(3)
    return driver

def quit(driver):
    """This method will diconect the selenium web driver and closing the chrome windows
       Parameters:
       Returns:
       Example:"""
    print("Fishing the selenium web driver")
    driver.quit()
    
def page_Ready(driver):
    """Method uses to wxecute some jquery code to wait until the page is ready
       Parameters:
       Returns:
       Example:"""
    page_state = driver.execute_script('return document.readyState;')
    return page_state
    
def is_Ready(driver):
    """The page ready method, make infinite loop to wait until web page has been downladed.
       Parameters:
       Returns:
       Example:"""
    while True:
        page_state = page_Ready(driver)
        if page_state == 'complete':
            print("Page ready...")
            break
            
def tab_manage(driver,num):
    """Method used to handdle the tabs, moving through them
       Parameters:
       Returns:
       Example:"""
    try:
        driver.switch_to.window(driver.window_handles[num])
        wait(3)
    except:
        print("Error handdling the tabs ")
        
def tab_closer(driver):
    """Method used to close the currently tab
       Parameters:
       Returns:
       Example:"""
    try:
        driver.close()
    except:
        print("\x1b[31m\"Error so bad closing the tab \"\x1b[0m")

In [3]:
###
# Methods to create waits
###
def explicit_wait(driver,many,key,path_element,num):
    """This method is create to create a explicit wait
       Parameters:
       Returns:
       Example:"""
    dicc_Bys={'selector':By.CSS_SELECTOR,'id':By.ID,
              'tag':By.TAG_NAME,'xpath':By.XPATH}
    try:
        if many:
            element = WebDriverWait(driver,num).until(
                EC.presence_of_all_elements_located(
                    (dicc_Bys[key],path_element)
                )
            )
        else:
            element = WebDriverWait(driver,num).until(
                EC.presence_of_element_located(
                    (dicc_Bys[key],path_element)
                )
            )
    except:
        print("\x1b[31m\"Something bad happen with the explicit wait \"\x1b[0m")
    return element

def implicit_wait(driver,many,key,path_element,num):
    """Method used to handle a implicit wait:
       Parameters:
       Returns:
       Example:"""
    driver.implicitly_wait(num)
    try:
        if many:
            if key == 'id':
                element = driver.find_element_by_id(path_element)
            if key == 'selector':
                element = driver.find_elements_by_css_selector(path_element)
            if key ==  'xpath':
                element = driver.find_elements_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_elements_by_tag_name(path_element)
        else:
            if key == 'selector':
                element = driver.find_element_by_css_selector(path_element)
            if key == 'xpath':
                element = driver.find_element_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_element_by_tag_name(path_element)
    except:
        print("\x1b[31m\"Something bad happen with the implicit wait function \"\x1b[0m")
    return element

def hard_wait(driver,many,key,path_element,num):
    """This method create a system wait fro handdle in genereal all the Ajax interactions
       Parameters:
       Returns:
       Example:"""
    time.sleep(num)
    try:
        if many:
            if key == 'id':
                element = driver.find_element_by_id(path_element)
            if key == 'selector':
                element = driver.find_elements_by_css_selector(path_element)
            if key ==  'xpath':
                element = driver.find_elements_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_elements_by_tag_name(path_element)
        else:
            if key == 'selector':
                element = driver.find_element_by_css_selector(path_element)
            if key == 'xpath':
                element = driver.find_element_by_xpath(path_element)
            if key == 'tag':
                element = driver.find_element_by_tag_name(path_element)
    except:
        print("\x1b[31m\"Something bad happen with the hard wait function \"\x1b[0m")
    return element

def wait(num):
    """This method create a system wait fro handdle in genereal all the Ajax interactions
       Parameters:
       Returns:
       Example:"""
    time.sleep(num)
    return None

def waits_and_elements(driver,many,key,path_element,time,nombre):
    """Method used to handdle all waits
       Parameters:
       Returns:
       Example:"""
    try:
        elements = explicit_wait(driver,many,key,path_element,time)
    except:
        try:
            elements = implicit_wait(driver,many,key,path_element,time)
            if elements == []:
                elements = hard_wait(driver,many,key,path_element,time)
        except:
            print("\x1b[31m\"Something really bad happend, with: "+nombre+" "+path_element+", could be a change of the HTML DOM \"\x1b[0m")
    return elements

In [4]:
###
# Handdling buttons
### 
def random_translator(driver,segs):
    """Method used to handdle the button of translate a comment.
       Parameters:
       Returns:
       Example:"""
    str_div = 'div[class="ui_radio"]'
    elements = waits_and_elements(driver,True,'selector',str_div,segs,'botón_para_traducir')
    try:
        elements[-1].click()
    except:
        print("\x1b[31m\"Random translator is not here\"\x1b[0m")
        
def next_page(driver,segs):
    """Mehtod used to navigate across the next pages buttons
       Parameters:
       Returns:
       Example:"""
    str_next_b = 'a[class="nav next ui_button primary  cx_brand_refresh_phase2"]'
    try:
        element = waits_and_elements(driver,False,'selector',str_next_b,segs,'botón_para_cambiar_página')
        _make_click("NEXT PAGE",element,driver)
        is_Ready(driver)
        return True
    except:
        print("\x1b[31m\"There is no more pages to make click\"\x1b[0m")
        return False
        
def is_Lenguage(driver,segs):
    """Metod used to make click over the idiom botton
       Parameters:
       Returns:
       Example:"""
    try:
        str_button_es = 'input[id="filters_detail_language_filterLang_es"]'
        str_button_in = 'input[id="filters_detail_language_filterLang_en"]'
        objects_es = waits_and_elements(driver,False,'selector',str_button_es,segs,'Botón_español')
        objects_in = waits_and_elements(driver,False,'selector',str_button_es,segs,'Botón_inglés')
        diccionario = {
            objects_es.text.split('(')[0]:objects_es,
            objects_in.text.split('(')[0]:objects_in
        }   
    except:
        print("\x1b[31m\"Lenguages was not obtaned\"\x1b[0m")
    return diccionario

####I DO NOT KNOW WHY, BUT SHOW MORE ONLY WORKS AFTER YOU INTERACT WITH THE PAGE (MAKING CLICKS ETC ETC).
def show_more(driver):
    """Method use to make click over the comment area to show the full comment
       Parameters:
       Returns:
       Example:"""
    str_show_more = 'span[class="taLnk ulBlueLinks"]'
    elements = waits_and_elements(driver,False,'selector',str_show_more,10,'botón_ver_más')
    try:
        element.click()
    except:
        print("\x1b[31m\"Show more is not here...\"\x1b[0m")

In [60]:
###
# Methods to extract the general information about the hotel
###
def header_information(driver,segs):
    """This method will interact only with thw header of the page, just to get the information about the place
       Parameters:
       Returns:
       Example:"""
    import pdb; pdb.set_trace()
    str_header = 'taplc_resp_rr_top_info_rr_resp_0'
    try:
        element = waits_and_elements(driver,False,'id',str_header,segs,'contenedor_header')
        try: # Name
            str_name = 'h1[class="ui_header h1"]'
            name = waits_and_elements(element,False,'selector',str_name,segs,'nombre_del_hotel').text
        except:
            name = "Sin_nombre"
        try: # Direction
            str_span = 'div[class="is-hidden-mobile blEntry address  ui_link_container level_4"]'
            dirc = waits_and_elements(element,False,'selector',str_span,segs,'dirección_hotel').text
        except:
            dirc = "Sin_dirección"
        try: # Score
            str_scr = 'span[class*="ui_bubble_rating"]'
            scrt = waits_and_elements(driver,False,'selector',str_scr,segs,'califiacion_del_hotel').get_attribute('alt').split(' ')[0]
        except:
            scrt = "Sin_score"
        try: # Phone
            str_phon = 'div[class="blEntry phone"]'
            phne = waits_and_elements(element,False,'selector',str_phon,segs,'telefono_hotel').text
        except:
            phne = "Sin_teléfono"
        try:
            str_site = 'div[class="is-hidden-mobile blEntry website  ui_link_container level_1"]'
            wait(1)
            waits_and_elements(element,False,'selector',str_site,segs,'sitio_web_hotel').click()
            tab_manage(driver,-1)
            site = driver.current_url
            tab_closer(driver)
            tab_manage(driver,0)
        except:
            site = "Sin_sitio"
    except:
        print("\x1b[31m\"Exception: HEADER.Something bad happed with header_information, the element could no be located\"\x1b[0m")
    return name,dirc,scrt,phne,site

def header_information_v2(driver,segs):
    """Doc"""
    element = driver.find_element_by_css_selector('div[class="bk7Uv0cc"]')
    h1 = element.find_element_by_tag_name('h1')
    establecimiento = h1.text
    div = element.find_elements_by_css_selector('div[class="_1ud-0ITN"]')
    a_s = div[-1].find_elements_by_tag_name('a')
    l_data = []
    l_link = []
    for a in a_s:
        l_data.append(a.text)
        l_link.append(a.get_attribute('href'))
    l_link[0]=l_data[0]
    return establecimiento,l_link

In [6]:
###
# Methods to extract all the information about the comments
###
def all_comment(driver,idiom,tipo,segs):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    str_div = 'div[class="review-container"]'
    list_usuario = []
    list_perfil = []
    list_localidad = []
    list_fecha_comentario = []
    list_score = []
    list_titulo = []
    list_coment = []
    list_fecha_visita = []
    list_tipo = []
    list_idiom = []
    try:
        elements = waits_and_elements(driver,True,'selector',str_div,segs,'div_comentarios')
        for element in elements:
            #print(element.text)
            try: # User and Profile
                str_div = 'div[class="info_text pointer_cursor"]'
                object_ = waits_and_elements(element,False,'selector',str_div,segs,'usuario')
                user = object_.text
                str_div = 'div[class="info_text pointer_cursor"]'
                object_ = waits_and_elements(element,False,'selector',str_div,segs,'usuario')
                user = object_.text
                perfil = "Sin_perfil"
                ubicacion = "Sin_ubicación"
                ##MAKE CLICK TO GET THE USER AND LOCALITY
                #_make_click("USER AND LOCALITY",object_,driver)
                #try:
                #    str_inner = 'div[class="innerContent"]'
                #    inner_data = waits_and_elements(driver,False,'selector',str_inner,segs,'perfil-ubicación')
                #    perfil = waits_and_elements(inner_data,False,'tag','a',segs,'a_link').get_attribute('href')
                #    str_ul_info = 'ul[class="memberdescriptionReviewEnhancements"]'
                #    ubicacion = waits_and_elements(inner_data,False,'selector',str_ul_info,segs,'ul_ubicación').text
                #except:
                #    perfil = "Sin_perfil"
                #    ubicacion = "Sin_ubicación"
                #str_close = 'div[class="ui_close_x"]'
                #close = waits_and_elements(element,False,'selector',str_close,segs,'CLOSING')
                #_make_click("USER AND LOCALITY",close,driver)
            except:
                user = "Sin_usuario"
            try: # Date of comment
                str_fecha_coment = 'span[class="ratingDate"]'
                fecha = waits_and_elements(element,False,'selector',str_fecha_coment,segs,'fecha_del_comentario').get_attribute('title')
            except:
                fecha = "Sin_fecha_del_comentario"
            
            try: # Score
                str_div = 'div[class="ui_column is-9"]'
                abc = waits_and_elements(element,False,'selector',str_div,segs,'Califiación_usuario_hotel_1')
                score = waits_and_elements(abc,False,'tag','span',segs,'califiación_usuario_hotel_2').get_attribute('class')
                score = score.replace('ui_bubble_rating bubble_','')
            except:
                score = "Sin_calificacion" 
            
            try: # Header comment
                str_coment_titulo = 'span[class="noQuotes"]'
                header = waits_and_elements(element,False,'selector',str_coment_titulo,segs,'comentario_título').text
            except:
                header = "Sin_titulo"
            
            try:
                elmento=element.find_elements_by_css_selector('span[class="taLnk ulBlueLinks"]')
                #for itm in elmento:
                _make_click("USER AND LOCALITY",elmento,driver)
            except:
                print("\x1b[31m\"Clicking again show more \"\x1b[0m")
            
            try: #the show more
                str_show_more = 'p[class="partial_entry"]'
                texto = waits_and_elements(element,False,'selector',str_show_more,segs,'botón_ver_más').text
            except:
                texto = "Sin texto"
            
            try: # Date of visit
                str_date_visit = 'div[class="prw_rup prw_reviews_stay_date_hsx"]'
                date_of_visit = waits_and_elements(element,False,'selector',str_date_visit,segs,'fecha_visita').text.replace('Fecha de la visita: ','')
            except:
                date_of_visit = "Sin_fecha_de_visita"
            
            list_usuario.append(user)
            list_perfil.append(perfil)
            list_localidad.append(ubicacion)
            list_fecha_comentario.append(fecha)
            list_score.append(score)
            list_titulo.append(header)
            list_coment.append(texto)
            list_fecha_visita.append(date_of_visit)
            list_tipo.append(tipo)
            list_idiom.append(idiom)
    except StaleElementReferenceException as e:
        print("\x1b[31m\"Exception:BODY_COMMENTS Something bad happend with all_comment, the element could ni be located\"\x1b[0m")
    return list_usuario,list_perfil,list_localidad,list_fecha_comentario,list_score,list_titulo,list_coment,list_fecha_visita,list_tipo,list_idiom

In [7]:
###
# Methods to get the buttons diccs
###
def date_consult(is_finally=True):
    """Método utilizado para crear la fecha de consulta
       Parámetros: <None>
       Return: <date> tipo: date --> Fecha de salida del sistema"""
    if is_finally:
        now = datetime.datetime.now()
        now = now.strftime('%Y-%m-%d_%H-%M-%S')
    else:
        now = date.today()
        now = str(now)
    return now

def buttons(driver,segs):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    str_calif =  'div[class="prw_rup prw_filters_detail_checkbox ui_column separated is-5"]'
    str_tipo_epoc = 'div[class="prw_rup prw_filters_detail_checkbox ui_column separated is-2"]'
    str_idiom = 'div[class="prw_rup prw_filters_detail_language ui_column separated is-3"]'
    str_check = 'div[class="ui_checkbox item cx_brand_refresh_phase2"]'
    str_idm = 'div[class="item"]'
    
    dicc_calf = {}
    dicc_tipo = {}
    dicc_prdo = {}
    dicc_idms = {}
    
    big_item = waits_and_elements(driver,False,'selector',str_calif,segs,'Big_Calificación')
    addf = waits_and_elements(big_item,True,'selector',str_check,segs,'Botones_Calificación')
    for item in addf:
        if len(item.text.split(" ")) > 2:
            dicc_calf["".join(item.text.split(" ")[0:2])] = item
        else:
            dicc_calf[item.text.split(" ")[0]] = item
    big_epoc_tipo = waits_and_elements(driver,False,'selector',str_tipo_epoc,segs,'Big_Tipo')
    filters_t_e = waits_and_elements(big_epoc_tipo,True,'selector',str_check,segs,'Filtros_Tipo')
    for item in filters_t_e:
        if ("-" in item.text):
            dicc_prdo[item.text.replace("-","_")] = item
        elif (" " in item.text) or (item.text ==""):
            pass
        else: 
            dicc_tipo[item.text] = item
    big_idiom = waits_and_elements(driver,False,'selector',str_idiom,segs,'Big_Idioma')
    aaaa = waits_and_elements(big_idiom,True,'selector',str_idm,segs,'Filtros_idioma')
    for item in aaaa:
        if len(item.text.split(" ")) < 3:
              dicc_idms[item.text.split(" ")[0]] = item
    dicc_idms.pop("")
    return dicc_calf,dicc_tipo,dicc_prdo,dicc_idms

def _make_click(k,v,driver):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    try:
        v.click()
    except StaleElementReferenceException as e: 
        wait(2)
        driver.execute_script('arguments[0].click()', v)
    except:
        print("Error making click over: ",k)
    wait(2)
    
def _make_click_buttons(k,driver,tipo,cali,idmm,prdo):
    """"""
    if tipo:
        _,dicc_tipo,_,_=buttons(driver,0.5)
        _make_click(k,dicc_tipo[k],driver)
    if cali:
        dicc_calf,_,_,_=buttons(driver,0.5)
        _make_click(k,dicc_calf[k],driver)
    if prdo:
        _,_,dicc_prdo,_=buttons(driver,0.5)
        _make_click(k,dicc_prdo[k],driver)
    if idmm:
        _,_,_,dicc_idms=buttons(driver,0.5)
        _make_click(k,dicc_idms[k],driver)
    
def save(df,name,date,filter_=False,describe=False,mode_append=True):
    """Método para guardar el df resultante
       Parámetros: <df>   tipo: Pandas.DataFrame --> El dataframe con la información
                   <name> tipo: str --> El nombre del establesimiento
                   <date> tipo: datime.date --> La fecha en la que se realizo la consulta
       Retorno: <None>"""
    if describe:
        df[['Filtro_1','Filtro_2','Filtro_3']] = df.Categoria.str.split("_",expand=True) 
    if " " in name: name = name.replace(" ","_")
    if filter_: name = name+"_"+"filtered"
    if not mode_append:
        df.to_csv(name+"_"+date+'_pipes.csv',sep='|',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
        df.to_csv(name+"_"+date+'_comma.csv',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
        print("Datos correctamente guardados")
    else:
        df.to_csv(name+"_"+date+'_pipes.csv',sep='|',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig",mode='a')
        df.to_csv(name+"_"+date+'_comma.csv',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig",mode='a')
    return None

def _a_button(driver,segs,dicc,*argv):
    """Method used to get all the coments and all the information inside of them
       Parameters:
       Returns:
       Example:"""
    str_a = 'a[class="pageNum last   cx_brand_refresh_phase2"]'
    as_ = waits_and_elements(driver,True,'selector',str_a,segs,'As')
    if as_ != []:
        num = int(as_[-1].text)*10
        _make_click('a_last',as_[-1],driver)
    else:
        num = 0
    str_divs = 'div[class="rev_wrap ui_columns is-multiline"]'
    divs_ = waits_and_elements(driver,True,'selector',str_divs,segs,'DIVs')
    if divs_ != []:
        num = num + len(divs_)
    else:
        num = 0
    dicc['Categoria'].append(argv[0]+'_'+argv[1]+'_'+argv[2])
    dicc['Contador_Comentarios'].append(num)
    return dicc

In [62]:
###
# Run all
###
def run_all(url,name_file,esp=True,eng=True,max_page=None,segs=0.5):
    if max_page == None: iterator = 10000
    else: iterator = max_page
    dicc_data = {
        'Establecimiento':[],
        'Direccion':[],
        'Telefono':[],
        'Sitio':[],
        'Menu':[],
        'Usuario':[],
        'Perfil':[],
        'Localidad':[],
        'Fecha_comentario':[],
        'Califiacion_persona':[],
        'Titulo':[],
        'Comentario':[],
        'Fecha_visita':[],
        'Tipo_comensal':[],
        'Idioma':[]
    }
    df = pd.DataFrame(dicc_data)
    driver = connect(url)
    is_Ready(driver)
    name,lista = header_information_v2(driver,segs)
    _,dicc_tipo,_,dicc_idms=buttons(driver,segs)
    for k_i,v_i in dicc_idms.items():
        print(k_i)
        is_Ready(driver)
        _,_,_,dicc_idms=buttons(driver,segs)
        _make_click(k_i,dicc_idms[k_i],driver)
        for k_t,v_t in dicc_tipo.items():
            print(k_t)
            is_Ready(driver)
            _,dicc_tipo,_,_=buttons(driver,segs)
            _make_click(k_t,dicc_tipo[k_t],driver)
            aux = iterator #10000
            while aux > 0:
                list_usuario,list_perfil,list_localidad,list_fecha_comentario,list_score,list_titulo,list_coment,list_fecha_visita,list_tipo,list_idiom = all_comment(driver,k_i,k_t,segs)
                for indx in range(len(list_usuario)):
                    dicc_data = {
                        'Establecimiento':name,
                        'Direccion':lista[0],
                        'Telefono':lista[1],
                        'Sitio':lista[2],
                        'Menu':lista[3],
                        'Usuario':list_usuario[indx],
                        'Perfil':list_perfil[indx],
                        'Localidad':list_localidad[indx],
                        'Fecha_comentario':list_fecha_comentario[indx],
                        'Califiacion_persona':list_score[indx],
                        'Titulo':list_titulo[indx],
                        'Comentario':list_coment[indx],
                        'Fecha_visita':list_fecha_visita[indx],
                        'Tipo_comensal':list_tipo[indx],
                        'Idioma':list_idiom[indx]
                    }
                    print(dicc_data)
                    df = df.append(dicc_data,ignore_index=True)
                save(df,name_file,date_consult(is_finally=False),mode_append=True)
                aux -= 1
                try:
                    next_page = driver.find_element_by_css_selector('a[class="nav next ui_button primary  cx_brand_refresh_phase2"]')
                    _make_click("NEXT PAGE",next_page,driver)
                    print("Me fui a la siguiente página")
                except:
                    break
                #if next_page(driver,segs) == False:
                #    break
            _,dicc_tipo,_,_=buttons(driver,segs)
            _make_click(k_t,dicc_tipo[k_t],driver)
    save(df,name_file,date_consult(is_finally=True),mode_append=False)
    quit(driver)

In [9]:
###
# Methos used to generate the descriptive information 
###
def run_descriptivos(url,name,segs):
    """"""
    driver = connect(url)
    is_Ready(driver)
    try:
        dicc = {"Categoria":[],
                "Contador_Comentarios":[]}
        df = pd.DataFrame(dicc)
        dicc_calf,dicc_tipo,_,dicc_idms=buttons(driver,0.5)
        for k_i,v_i in dicc_idms.items():
            print(k_i)
            is_Ready(driver)
            _,_,_,dicc_idms=buttons(driver,0.5)
            _make_click(k_i,dicc_idms[k_i],driver)
            is_Ready(driver)
            dicc = _a_button(driver,segs,dicc,k_i,"","")
            for k_c,v_c in dicc_calf.items():
                print("Primer click for calificacion: ",k_c)
                is_Ready(driver)
                dicc_calf,_,_,_=buttons(driver,0.5)
                print("Diccionario_ calificación 1: ",dicc_calf)
                _make_click(k_c,dicc_calf[k_c],driver)
                is_Ready(driver)
                dicc = _a_button(driver,segs,dicc,k_i,k_c,"")
                for k_t,v_t in dicc_tipo.items():
                    print("Primer click for tipo: ",k_t)
                    is_Ready(driver)
                    _,dicc_tipo,_,_=buttons(driver,0.5)
                    _make_click(k_t,dicc_tipo[k_t],driver)
                    dicc = _a_button(driver,segs,dicc,k_i,k_c,k_t)
                    is_Ready(driver)
                    print("Segundo click for tipo: ",k_t)
                    _,dicc_tipo,_,_=buttons(driver,0.5)
                    _make_click(k_t,dicc_tipo[k_t],driver)
                    is_Ready(driver)
                is_Ready(driver)
                print("Segundo click for calificacion: ",k_c)
                dicc_calf,dicc_tipo,dicc_prdo,dicc_idms=buttons(driver,0.5)
                print("Diccionario_ calificación 2: \n",dicc_calf,"\n",dicc_tipo,"\n",dicc_prdo,"\n",dicc_idms)
                _make_click(k_c,dicc_calf[k_c],driver)
                is_Ready(driver)
    except:
        print("\x1b[31m\"Error making click. CLICKS\"\x1b[0m")
    df = pd.DataFrame(dicc)
    save(df,name,date_consult(is_finally=True),mode_append=False)
    quit(driver)

<div class="alert alert-block alert-info">
<h3><center>Descriptivo</center></h3>
</div>

In [10]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): ")
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 0.5
else:
    wait_ = float(wait_)
run_descriptivos(url,name=name,segs=wait_)

Ingresa la url del sitio al que vamos a iterar: https://www.tripadvisor.com.mx/Restaurant_Review-g150781-d929237-Reviews-El_Farallon-Culiacan_Pacific_Coast.html
Ingresa el nombre de tu archivo: desc_30
Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): 2
Page ready...
español
Page ready...
Page ready...
Primer click for calificacion:  Excelente
Page ready...
Diccionario_ calificación 1:  {'Excelente': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="1de35632-559c-44f9-9d3e-e92edcede950")>, 'Muybueno': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="af69f6d0-3c59-4dd9-ba1d-369ffe2cc187")>, 'Regular': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="6f6cd5cf-a558-4e3b-9fa9-86cbc81387d2")>, 'Malo': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="

Page ready...
"Something bad happen with the explicit wait "
Primer click for tipo:  Familias
Page ready...
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Familias
Page ready...
Primer click for tipo:  Pareja
Page ready...
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Pareja
Page ready...
Primer click for tipo:  Solitario
Page ready...
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Solitario
Page ready...
Primer click for tipo:  Ejecutivo
Page ready...
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Ejecutivo
Page ready...
Primer click for tipo:  Amigos
Page ready...
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Amigos
Page ready...
Page ready...
Segundo click for calificacion:  Regular
Diccionario_ calificación 2: 
 {'Excelente': <selenium.webdriver.remote.webelement.WebElement (session="dd68

"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Ejecutivo
Page ready...
Primer click for tipo:  Amigos
Page ready...
"Something bad happen with the explicit wait "
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Amigos
Page ready...
Page ready...
Segundo click for calificacion:  Pésimo
Diccionario_ calificación 2: 
 {'Excelente': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="c540ab24-880e-40bb-b7f4-c28d83bfb714")>, 'Muybueno': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="209f0152-6216-4f72-a39c-924604526e6e")>, 'Regular': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="3c9ea5a3-5e62-4bc8-958a-8b7d513d9e6f")>, 'Malo': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="9a9758c3-e964-4e4a-ac44-cb

Page ready...
Primer click for calificacion:  Regular
Page ready...
Diccionario_ calificación 1:  {'Excelente': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="7ced8fd3-ac67-4f4e-b3a3-1b89b6bcdd67")>, 'Muybueno': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="8dac4d66-7d90-47fe-97bf-6dca904d92fb")>, 'Regular': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="a918dffe-afe8-4bf9-9409-5ce5833c8e52")>, 'Malo': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="16b294ef-3eac-400e-95a4-d202048cc4a0")>, 'Pésimo': <selenium.webdriver.remote.webelement.WebElement (session="dd68643c6aa111cf99d310e27c09a94e", element="62e03dcf-2de4-45f8-a8a6-84eef4f0626c")>}
Page ready...
"Something bad happen with the explicit wait "
"Something bad happen with the explicit wait "
Primer click for ti

Page ready...
"Something bad happen with the explicit wait "
"Something bad happen with the explicit wait "
Primer click for tipo:  Familias
Page ready...
"Something bad happen with the explicit wait "
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Familias
Page ready...
Primer click for tipo:  Pareja
Page ready...
"Something bad happen with the explicit wait "
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Pareja
Page ready...
Primer click for tipo:  Solitario
Page ready...
"Something bad happen with the explicit wait "
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Solitario
Page ready...
Primer click for tipo:  Ejecutivo
Page ready...
"Something bad happen with the explicit wait "
"Something bad happen with the explicit wait "
Page ready...
Segundo click for tipo:  Ejecutivo
Page ready...
Primer click for tipo:  Amigos
Page ready...
"Something bad happen with the expl

<div class="alert alert-block alert-info">
<h3><center>Idioma español u inglés o ambos</center></h3>
</div>

In [63]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = float(input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): "))
max_page = input("Ingresa el número máximo de páginas a iterar: ")
if max_page == "":
    max_page=None
else: max_page = int(max_page)
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 0.5
else:
    wait_ = float(wait_)
run_all(url,name,esp=True,eng=True,max_page=max_page,segs=wait_)

Ingresa la url del sitio al que vamos a iterar: https://www.tripadvisor.com.mx/Restaurant_Review-g150781-d929237-Reviews-El_Farallon-Culiacan_Pacific_Coast.html
Ingresa el nombre de tu archivo: farfa_1
Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): 2
Ingresa el número máximo de páginas a iterar: 500
Page ready...
español
Page ready...
Familias
Page ready...
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
"Clicking again show more "
{'Establecimiento': 'El Farallon', 'Direccion': 'Malecon Nuevo, Culiacán 80060 México', 'Telefono': 'tel:+52 667 258 1515', 'Sitio': 'http:

Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
"Something bad happen with the explicit wait "
"Something really bad happend, with: comentario_título span[class="noQuotes"], could be a change of the HTML DOM "
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
{'Establecimiento': 'El Farallon', 'Direccion': 'Malecon Nuevo, Culiacán 80060 México', 'Telefono': 'tel:+52 667 258 1515', 'Sitio': 'http://www.farallon.com.mx/', 'Menu': 'http://farallon.com.mx/menu.html', 'Usuario': 'Flyer740047', 'Perfil': 'Sin_perfil', 'Localidad': 'Sin_ubicación', 'Fecha_comentario': '15 de enero de 2020', 'Califiacion_persona': '50', 'Titulo': 'Culiacan', 'Comentario': 'Excelente opción para convivir, la comida, las bebidas y la atención hacia el cliente son de primera cali

Ejecutivo
Page ready...
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
{'Establecimiento': 'El Farallon', 'Direccion': 'Malecon Nuevo, Culiacán 80060 México', 'Telefono': 'tel:+52 667 258 1515', 'Sitio': 'http://www.farallon.com.mx/', 'Menu': 'http://farallon.com.mx/menu.html', 'Usuario': 'edgarc772', 'Perfil': 'Sin_perfil', 'Localidad': 'Sin_ubicación', 'Fecha_comentario': '13 de marzo de 2019', 'Califiacion_persona': '40', 'Titulo': 'Muy buena opción en mariscos', 'Comentario': 'Una alternativa para comer mariscos bastante buena, ideal para comidas de negocios o convivencias grupales, buena atenció

{'Establecimiento': 'El Farallon', 'Direccion': 'Malecon Nuevo, Culiacán 80060 México', 'Telefono': 'tel:+52 667 258 1515', 'Sitio': 'http://www.farallon.com.mx/', 'Menu': 'http://farallon.com.mx/menu.html', 'Usuario': 'Jorge V', 'Perfil': 'Sin_perfil', 'Localidad': 'Sin_ubicación', 'Fecha_comentario': '19 de febrero de 2015', 'Califiacion_persona': '50', 'Titulo': 'Mariscos frescos en Culiacan', 'Comentario': 'La variedad de mariscos es muy buena, y puedes comer y hacer negocios ya que nos es ruidoso, los camarones en aguachile son excelentes, asi como los ostiones en su concha, el cayo de hacha inmejorable, la atencion es buena con amplio estacionamiento.', 'Fecha_visita': 'febrero de 2015', 'Tipo_comensal': 'Ejecutivo', 'Idioma': 'español'}
{'Establecimiento': 'El Farallon', 'Direccion': 'Malecon Nuevo, Culiacán 80060 México', 'Telefono': 'tel:+52 667 258 1515', 'Sitio': 'http://www.farallon.com.mx/', 'Menu': 'http://farallon.com.mx/menu.html', 'Usuario': 'juancarlosg323', 'Perfil':

Pareja
Page ready...
"Something bad happen with the explicit wait "
Solitario
Page ready...
"Something bad happen with the explicit wait "
Ejecutivo
Page ready...
"Something bad happen with the explicit wait "
Amigos
Page ready...
Error making click over:  USER AND LOCALITY
Error making click over:  USER AND LOCALITY
{'Establecimiento': 'El Farallon', 'Direccion': 'Malecon Nuevo, Culiacán 80060 México', 'Telefono': 'tel:+52 667 258 1515', 'Sitio': 'http://www.farallon.com.mx/', 'Menu': 'http://farallon.com.mx/menu.html', 'Usuario': 'MRodriguez64', 'Perfil': 'Sin_perfil', 'Localidad': 'Sin_ubicación', 'Fecha_comentario': '11 de mayo de 2015', 'Califiacion_persona': '50', 'Titulo': 'Great Food', 'Comentario': 'Food is great but the Restaurant looks a little bit old. They change its name from El Farallon to Mar and Sea.', 'Fecha_visita': 'mayo de 2015', 'Tipo_comensal': 'Amigos', 'Idioma': 'inglés'}
{'Establecimiento': 'El Farallon', 'Direccion': 'Malecon Nuevo, Culiacán 80060 México', 'T

In [61]:
header_information_v2(driver,2)

('El Farallon',
 ['Malecon Nuevo, Culiacán 80060 México',
  'tel:+52 667 258 1515',
  'http://www.farallon.com.mx/',
  'http://farallon.com.mx/menu.html'])

In [58]:
driver = connect("https://www.tripadvisor.com.mx/Restaurant_Review-g150781-d929237-Reviews-El_Farallon-Culiacan_Pacific_Coast.html")

In [60]:
all_comment(driver,"Solitario",0.5)

Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "
Error making click over:  USER AND LOCALITY
"Clicking again show more "


(['Flyer740047',
  'Xavier A',
  'gabrielavalenzuela99',
  'Karisma S',
  '305arturod',
  'vsauceda2018',
  'aidualcma',
  'Malanco',
  'diegoimaz',
  'edgarc772\nGuadalajara, México'],
 ['Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil'],
 ['Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación'],
 ['15 de enero de 2020',
  '16 de agosto de 2019',
  '27 de diciembre de 2019',
  '1 de noviembre de 2019',
  '8 de octubre de 2019',
  '7 de agosto de 2019',
  '7 de agosto de 2019',
  '17 de julio de 2019',
  '15 de junio de 2019',
  '13 de marzo de 2019'],
 ['50', '40', '50', '50', '30', '50', '50', '40', '50', '40'],
 ['Culiacan',
  'Alta cocina con un toque regional y productos del mar muy frescos.',
  'Delicioso!',
  'Excelente lugar en Culiacán, muy r

In [37]:
div = element.find_elements_by_css_selector('div[class="_1ud-0ITN"]')
a_s = a_s[-1].find_elements_by_tag_name('a')
for a in a_s:
    print(a.text)
    print(a.get_attribute('href'))

Alfonso Reyes 66, Ciudad de México México
https://www.tripadvisor.com.mx/Restaurant_Review-g150800-d815301-Reviews-El_Diez-Mexico_City_Central_Mexico_and_Gulf_Coast.html#MAPVIEW
5553 0734
tel:5553 0734
Página web
http://www.eldiez.com.mx/
Menú
http://www.eldiez.com.mx/index#menu


In [38]:
nex_page = driver.find_element_by_css_selector('a[class="nav next ui_button primary  cx_brand_refresh_phase2"]')

In [39]:
nex_page.click()

207 opiniones
https://www.tripadvisor.com.mx/Restaurant_Review-g150800-d815301-Reviews-El_Diez-Mexico_City_Central_Mexico_and_Gulf_Coast.html#REVIEWS
#590 de 5,014 restaurantes en Ciudad de México
https://www.tripadvisor.com.mx/Restaurants-g150800-Mexico_City_Central_Mexico_and_Gulf_Coast.html
$$ - $$$
https://www.tripadvisor.com.mx/Restaurants-g150800-zfp10955-Mexico_City_Central_Mexico_and_Gulf_Coast.html
Churrasquería
https://www.tripadvisor.com.mx/Restaurants-g150800-c37-Mexico_City_Central_Mexico_and_Gulf_Coast.html
Latina
https://www.tripadvisor.com.mx/Restaurants-g150800-c10639-Mexico_City_Central_Mexico_and_Gulf_Coast.html
Fusión
https://www.tripadvisor.com.mx/Restaurants-g150800-c17-Mexico_City_Central_Mexico_and_Gulf_Coast.html
Alfonso Reyes 66, Ciudad de México México
https://www.tripadvisor.com.mx/Restaurant_Review-g150800-d815301-Reviews-El_Diez-Mexico_City_Central_Mexico_and_Gulf_Coast.html#MAPVIEW
5553 0734
tel:5553 0734
Página web
http://www.eldiez.com.mx/
Menú
http://w

In [40]:
all_comment(driver,"Solitario",0.5)

"Clicking again show more "
"Clicking again show more "
"Clicking again show more "
"Clicking again show more "
"Clicking again show more "
"Clicking again show more "
"Clicking again show more "
"Clicking again show more "
"Clicking again show more "
"Clicking again show more "


(['AGMX1',
  'Estefania F',
  'jez_mx',
  'Claucparada2',
  'R13R',
  'JoseCarlosSanchezV\nMexicali, México',
  'nadiamG1995YF',
  'tanichi\nCuliacán, México',
  'Cecilia L',
  'josecr679'],
 ['Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil',
  'Sin_perfil'],
 ['Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación',
  'Sin_ubicación'],
 ['27 de enero de 2019',
  '15 de enero de 2019',
  '30 de noviembre de 2018',
  '14 de noviembre de 2018',
  '7 de noviembre de 2018',
  '23 de octubre de 2018',
  '10 de agosto de 2018',
  '7 de agosto de 2018',
  '6 de agosto de 2018',
  '17 de abril de 2018'],
 ['50', '40', '50', '50', '50', '20', '50', '50', '40', '50'],
 ['Excelente Calidad',
  'Buen servicio, agradable lugar...',
  'Increible lugar',
  'Excelente Mariscos y servicio',
  'Must!',


<div class="alert alert-block alert-info">
<h3><center>Idioma solo español</center></h3>
</div>

In [17]:
element.text

'El Farallon\nSolicitado\n61 opiniones#5 de 225 restaurantes en Culiacán$$ - $$$Mariscos\nMalecon Nuevo, Culiacán 80060 México+52 667 258 1515Página webMenú\nCerrado ahora: Ver todos los horarios'

In [ ]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = float(input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): "))
max_page = int(input("Ingresa el número máximo de páginas a iterar: "))
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 0.5
else:
    wait_ = float(wait_)
run_all(url,name,es,max_page=max_page,segs=wait_)

<div class="alert alert-block alert-info">
<h3><center>Idioma solo inglés</center></h3>
</div>

In [ ]:
url = input("Ingresa la url del sitio al que vamos a iterar: ")
name = input("Ingresa el nombre de tu archivo: ")
wait_ = float(input("Ingresa el número de segundos que deseas esperar por un elemento (ejemplo: 10): "))
max_page = int(input("Ingresa el número máximo de páginas a iterar: "))
if name == "":
    name= '_'
if wait_ == "":
    wait_ = 10
run_all_en(url,name,max_page=max_page,segs=wait)